In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup as soup 
import re
import pandas as pd
import numpy as np
import time
import json
import random

In [ ]:
browser = Browser('chrome')
city = "mumbai"
target_cars = 5000
cars_collected = 0
total_pages = 250 

In [ ]:
# 1. Open a text file in 'a' (append) mode
with open("car_links.txt", "a") as f:

    for page_num in range(1, total_pages + 1):
        if page_num == 1:
            url = f"https://www.cardekho.com/used-cars+in+{city}"
        else:
            url = f"https://www.cardekho.com/used-cars+in+{city}/page-{page_num}"
        
        browser.visit(url)
        time.sleep(2) 
        
        browser.execute_script("window.scrollTo(0, 1000);")
        time.sleep(1)

        current_soup = soup(browser.html, 'html.parser')
        
        page_links_found = 0
        for link in current_soup.find_all('a', href=True):
            href = link['href']
            
            if 'used-car-details' in href:
                # Build the full URL
                full_url = f"https://www.cardekho.com{href}" if href.startswith('/') else href
                
                # 2. Write the link to the text file immediately
                f.write(full_url + "\n")
                
                cars_collected += 1
                page_links_found += 1

        print(f"Page {page_num}: Saved {page_links_found} links. Total: {cars_collected}")

        # Stop if target reached
        if cars_collected >= target_cars:
            print("Target 5000 reached!")
            break

print("All links are now saved in car_links.txt")

Page 1: Saved 28 links. Total: 28
Page 2: Saved 28 links. Total: 56
Page 3: Saved 28 links. Total: 84
Page 4: Saved 28 links. Total: 112
Page 5: Saved 28 links. Total: 140
Page 6: Saved 28 links. Total: 168
Page 7: Saved 28 links. Total: 196
Page 8: Saved 28 links. Total: 224
Page 9: Saved 28 links. Total: 252
Page 10: Saved 28 links. Total: 280
Page 11: Saved 28 links. Total: 308
Page 12: Saved 28 links. Total: 336
Page 13: Saved 28 links. Total: 364
Page 14: Saved 28 links. Total: 392
Page 15: Saved 28 links. Total: 420
Page 16: Saved 28 links. Total: 448
Page 17: Saved 28 links. Total: 476
Page 18: Saved 28 links. Total: 504
Page 19: Saved 28 links. Total: 532
Page 20: Saved 28 links. Total: 560
Page 21: Saved 28 links. Total: 588
Page 22: Saved 28 links. Total: 616
Page 23: Saved 28 links. Total: 644
Page 24: Saved 28 links. Total: 672
Page 25: Saved 28 links. Total: 700
Page 26: Saved 28 links. Total: 728
Page 27: Saved 28 links. Total: 756
Page 28: Saved 28 links. Total: 784
Page

## Main Extraction 

In [ ]:
def get_browser():
    return Browser('chrome')

# 1. Load your links
with open("car_links.txt", "r") as f:
    all_links = [line.strip() for line in f.readlines()]

output_file = f"car_dataset_{city}.json"
browser = get_browser()

# 2. Loop through each link
for index, link in enumerate(all_links):
    try:
        print(f"Scraping {index+1}/{len(all_links)}: {link}")
        
        # Visit the link once for all data
        browser.visit(link)
        
        # Human-like delay and scroll to trigger dynamic content
        time.sleep(random.uniform(1, 2))
        browser.execute_script("window.scrollTo(0, 600);")
        time.sleep(0.5)

        # 3. Expansion Logic (Clicking 'View all Specifications')
        try:
            view_all_spec_btn = browser.find_by_text('View all Specifications')
            if view_all_spec_btn:
                browser.execute_script("arguments[0].click();", view_all_spec_btn.first._element)
                print("Expanded specifications.")
                time.sleep(0.6) # Wait for table to render
        except Exception:
            # Skip if button is missing or already expanded
            pass

        # 4. Scrape with BeautifulSoup
        page_soup = soup(browser.html, 'html.parser')
        car_data = {"url": link}
        
        # --- PRICE EXTRACTION ---
        price_div = page_soup.find('div', class_='vehiclePrice')
        if price_div:
            price_span = price_div.find('span')
            if price_span:
                car_data["Price"] = price_span.get_text(strip=True)

        # --- SPECIFICATIONS EXTRACTION ---
        spec_items = page_soup.find_all('li', class_='gsc_col-xs-12')
        for item in spec_items:
            label_tag = item.find('div', class_='label')
            value_tag = item.find('span', class_='value-text')
            if label_tag and value_tag:
                label = label_tag.get_text(strip=True)
                value = value_tag.get_text(strip=True)
                car_data[label] = value

        # 5. Save if valid data was found
        if len(car_data) > 1:
            with open(output_file, "a") as out:
                out.write(json.dumps(car_data) + "\n")
            print(f"Saved: {car_data.get('Price', 'N/A')} and {len(car_data)-2} specs.")
        else:
            print(f"No data found for: {link}")

    except Exception as e:
        print(f"Serious error at {link}: {e}")
        # Refresh browser instance on crash
        browser.quit()
        browser = get_browser()
        time.sleep(1)
        continue

browser.quit()

Scraping 1/6024: https://www.cardekho.com/used-car-details/used-Mini-cooper-countryman-2018-2021-cooper-s-jcw-inspired-cars-Bangalore_18ca46db-90fa-4cf1-b4ba-db8f05a5e85e.htm?adId=16580&adType=41
Expanded specifications.
Saved: ₹33 Lakh and 51 specs.
Scraping 2/6024: https://www.cardekho.com/used-car-details/used-Hyundai-i20-sportz-12-cars-Bangalore_7df4f53f-c332-407b-bf22-8eafa5c96570.htm
Expanded specifications.
Saved: ₹6.10 Lakh and 52 specs.
Scraping 3/6024: https://www.cardekho.com/used-car-details/used-Hyundai-i20-asta-opt-bsvi-cars-Bangalore_b13dbf1d-b492-4e05-8298-0ec94c66c580.htm
Expanded specifications.
Saved: ₹7.25 Lakh and 44 specs.
Scraping 4/6024: https://www.cardekho.com/used-car-details/used-Volkswagen-vento-12-tsi-highline-at-cars-Bangalore_013fc818-a77b-4d18-bcf1-da70117f1d26.htm
Expanded specifications.
Saved: ₹6.85 Lakh and 52 specs.
Scraping 5/6024: https://www.cardekho.com/used-car-details/used-Volvo-s60-d4-momentum-bsiv-cars-Bangalore_68cc26f5-52b3-4124-b684-9af3

KeyboardInterrupt: 

In [ ]:
def convert_json_to_cleaned_csv(input_json, output_csv):
    try:
        # 1. Load JSON Lines into a DataFrame
        df = pd.read_json(input_json, lines=True)
        
        if df.empty:
            print(f"No data found in {input_json}")
            return

        # 2. Clean Column Names
        # Strips whitespace and ensures headers are clean for your dataset
        df.columns = [col.strip() for col in df.columns]


        # 3. Save to CSV
        # index=False removes the unnecessary row numbers
        # encoding='utf-8-sig' ensures Indian currency symbols (₹) display correctly in Excel
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        
        print(f"Successfully converted {len(df)} rows to {output_csv}")

    except Exception as e:
        print(f"Error during conversion: {e}")

# Usage
convert_json_to_cleaned_csv(f'car_dataset{city}.json', f'car_dataset_{city}.csv')

Successfully converted 5544 rows to car_dataset_banglore.csv


---

In [ ]:
base_url = "https://www.cardekho.com/used-car-details/used-Toyota-urban-cruiser-premium-at-cars-Thane_fac1dc99-bc79-434f-b5fc-9834e1928e0b.htm?adId=17824&adType=41"

In [17]:
# Visit the website
browser.visit(base_url)

# To get more result scroll down the browser
scroll_count = 4
scroll_delay = 2

# 1. Scroll down to ensure the "Specifications" section is loaded
browser.execute_script("window.scrollTo(0, 1000);")
time.sleep(2)

# 2. Find and click the button once
try:
    # Finding the 'View all Specifications' button
    view_all_spec_btn = browser.find_by_text('View all Specifications')
    
    if view_all_spec_btn:
        # Using a JavaScript click to ensure it bypasses any overlays
        browser.execute_script("arguments[0].click();", view_all_spec_btn.first._element)
        time.sleep(2) 
        print("Successfully expanded the specifications.")
    else:
        print("Button not found. It might already be expanded or the text changed.")

except Exception as e:
    print(f"An error occurred: {e}")


Successfully expanded the specifications.


In [18]:
html = browser.html
page_soup = soup(html, 'html.parser')
spec_items = page_soup.find_all('li', class_='gsc_col-xs-12')

specs_data = {}

for item in spec_items:
    label_tag = item.find('div', class_='label')
    # Find the value (e.g., "1798 cc")
    value_tag = item.find('span', class_='value-text')
    
    if label_tag and value_tag:
        label = label_tag.get_text(strip=True)
        value = value_tag.get_text(strip=True)
        specs_data[label] = value

# Print a few to verify
print(f"Scraped {len(specs_data)} specifications.")
print(f"Engine: {specs_data.get('Engine')}")

Scraped 41 specifications.
Engine: 1462 cc


In [19]:
specs_data

{'Registration Year': 'Dec 2021',
 'Insurance': 'Comprehensive',
 'Fuel Type': 'Petrol',
 'Seats': '5 Seats',
 'Kms Driven': '28,476 Kms',
 'RTO': 'Thane',
 'Ownership': 'First Owner',
 'Engine Displacement': '1462 cc',
 'Transmission': 'Automatic',
 'Year of Manufacture': '2021',
 'Engine': '1462 cc',
 'Power': '103.26 bhp',
 'Mileage': '18.76 kmpl',
 'Fuel Type': 'Petrol',
 'No. of Airbags': '2',
 'Engine Type': 'k-series1.5l',
 'Displacement': '1462 cc',
 'Max Power': '103.26bhp@6000rpm',
 'Max Torque': '138nm@4400rpm',
 'No. of Cylinders': '4',
 'Valves Per Cylinder': '4',
 'Transmission Type': 'Automatic',
 'Gearbox': '4 Speed',
 'Petrol Mileage ARAI': '18.76 kmpl',
 'Petrol Fuel Tank Capacity': '48 Litres',
 'Emission Norm Compliance': 'BS VI',
 'Front Suspension': 'McPherson Strut with Coil Spring',
 'Rear Suspension': 'Torsion Beam with Coil Spring',
 'Steering Type': 'Electronic',
 'Steering Column': 'Tilt',
 'Turning Radius': '5.2',
 'Front Brake Type': 'Ventilated Disc',
 'R